### Look into "What are the payment methods preferred by social drinkers?"

from: https://dev-10.teachable.com/courses/data-engineering-c50/lectures/45590968

In [4]:
import pandas as pd
import os, sys
from IPython.display import display

### Load in all the csv's, explore the fields and shape for each

Use one dictionary , `dfs` so we don't need separate variable names for each df, but can still reference each one based off the filename we loaded it in from.

In [3]:
data_dir = '../data/restaurants/'
os.listdir(data_dir)
dfs = {}
for fn in os.listdir(data_dir):
    if fn.lower().endswith('.csv'):
        df_name = fn.split('.')[0]
        dfs[df_name] = pd.read_csv(os.path.join(data_dir, fn))


In [33]:
for k,v in dfs.items():
    print(k, v.shape)
    display(v.head(2))

chefmozaccepts (1314, 2)


,placeID,Rpayment
0,135110,cash
1,135110,VISA


chefmozcuisine (916, 2)


,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian


chefmozhours4 (2339, 3)


...

,userID,Rcuisine
0,U1001,American
1,U1002,Mexican


userpayment (177, 2)


,userID,Upayment
0,U1001,cash
1,U1002,cash


userprofile (138, 19)


,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87


In [34]:
# Since we need to connect "social drinkers" to "payment method" 
# we'll need separate df's, see next section
[(k, v.columns) for k,v in dfs.items()]

[('chefmozaccepts', Index(['placeID', 'Rpayment'], dtype='object')),
 ('chefmozcuisine', Index(['placeID', 'Rcuisine'], dtype='object')),
 ('chefmozhours4', Index(['placeID', 'hours', 'days'], dtype='object')),
 ('chefmozparking', Index(['placeID', 'parking_lot'], dtype='object')),
 ('geoplaces2',
  Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
         'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
         'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
         'area', 'other_services'],
        dtype='object')),
 ('rating_final',
  Index(['userID', 'placeID', 'rating', 'food_rating', 'service_rating'], dtype='object')),
 ('usercuisine', Index(['userID', 'Rcuisine'], dtype='object')),
 ('userpayment', Index(['userID', 'Upayment'], dtype='object')),
 ('userprofile',
  Index(['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
         'dress_preference', 'ambience', 'transport', 'marital_status',

### Select DataFrame: `userpayment`, `userprofile`

In [24]:
userpayment = dfs['userpayment']
userprofile = dfs['userprofile']

display(userpayment.head(5))
display(userprofile.head(5))

,userID,Upayment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash
4,U1004,bank_debit_cards


,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69


In [32]:
# get userID's of user registered as social drinkers
vc_drinks = userprofile['drink_level'].value_counts()
is_social = vc_drinks.index[2]
vc_drinks


abstemious        51
casual drinker    47
social drinker    40
Name: drink_level, dtype: int64

In [31]:
ind = userprofile['drink_level'] == is_social
social_drinkers = userprofile[ind][['userID']]
print(len(social_drinkers))
social_drinkers.head(3)

40


,userID
2,U1003
5,U1006
7,U1008


### Check shape and uniqueness of userID's

They are all unique in `userprofile` but they are repeated in `userpayment` meaning this table acts like the many side of a one-to-many of `userprofile(userID) -> userpayment(userID)`

ETL:
 - load csv
 - transform obj -> int64
 - load into matplotlib

In [28]:
len(social_drinkers['userID'].unique()), social_drinkers.shape

(40, (40, 1))

In [30]:
len(userpayment['userID'].unique()), userpayment.shape

(133, (177, 2))

### Perform the merge

In [35]:
len(social_drinkers), len(userpayment)

(40, 177)

In [36]:
both = pd.merge(social_drinkers, userpayment, how='inner', on='userID')


In [37]:
len(both)

56

In [40]:
both

,userID,Upayment
0,U1003,cash
1,U1006,cash
2,U1008,cash
3,U1010,cash
4,U1015,cash
5,U1021,cash
6,U1027,cash
7,U1033,cash
8,U1036,cash
9,U1037,cash


In [39]:
both['Upayment'].value_counts()

cash                   38
VISA                    8
bank_debit_cards        8
MasterCard-Eurocard     2
Name: Upayment, dtype: int64

In [ ]:
# So for social drinkers,
# 38 use cash, followed by 8 for visa.

In [41]:
# compared to all users
userpayment['Upayment'].value_counts()

cash                   131
bank_debit_cards        22
VISA                    17
MasterCard-Eurocard      4
American_Express         3
Name: Upayment, dtype: int64